# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID 19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis.Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da Sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD – Covid19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19

### Instalando dependências

In [ ]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #instalando java 8
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz #baixando Spark
!tar xf spark-3.1.2-bin-hadoop2.7.tgz #descompactando Spark
!pip install -q findspark

### Importando bibliotecas

In [ ]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
import os


os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark


findspark.init()

### Criando sessão Spark

In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master('local[*]')\
    .appName("Iniciando com Spark")\
    .getOrCreate()

In [16]:
spark

### Importando dados

In [17]:
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import zipfile

In [24]:
zipfile.ZipFile('/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/PNAD_COVID.zip', 'r').extractall('/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados')

In [55]:
path = '/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/PNAD_COVID.csv'
df = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [71]:
print(df.count(), df.count() // 5)

2650459 530091


### Exportando dados

In [57]:
#exportando seguimentos do DataFrame
n_splits = 5
n_rows = df.count() // n_splits
copy_df = df

for i in range(1, n_splits + 1):
    temp_df = copy_df.limit(n_rows)
    copy_df = copy_df.subtract(temp_df) #subtraindo conteúdo seguimentado

    #exportando seguimento
    temp_df.coalesce(1).write.csv(
    path=f'/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/seguimento_{i}',
    mode='overwrite',
    sep=',',
    header=True
    )

In [58]:
path = '/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/seguimento_1/*.csv'
df_1 = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [59]:
path = '/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/seguimento_2/*.csv'
df_2 = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [60]:
path = '/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/seguimento_3/*.csv'
df_3 = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [61]:
path = '/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/seguimento_4/*.csv'
df_4 = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [62]:
path = '/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/seguimento_5/*.csv'
df_5 = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [63]:
df_1.show(5)

+----+---+-------+-------+-----+-----+-----+-----+-------+---------+-----+-----+------+-----------+------------+------+----+-----+------+------+------+----+----+----+----+----+-----+-----+----+-----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+------+------+-----+----+----+-----+----+-----+------+------+-----+------+------+-----+-----+-----+------+-------+-------+------+-------+-------+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+------+------+------+------+------+-----+----+
| Ano| UF|CAPITAL|RM_RIDE|V1008|V1012|V1013|V1016|Estrato|      UPA|V1022|V1023| V1030|      

In [75]:
df_4.orderBy('V1013').show(5)

+----+---+-------+-------+-----+-----+-----+-----+-------+---------+-----+-----+-------+------------+------------+------+----+-----+------+------+------+----+----+----+----+----+-----+-----+----+-----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+------+------+-----+----+----+-----+----+-----+------+------+-----+------+------+-----+-----+-----+------+-------+-------+------+-------+-------+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+------+------+------+------+------+-----+----+
| Ano| UF|CAPITAL|RM_RIDE|V1008|V1012|V1013|V1016|Estrato|      UPA|V1022|V1023|  V1030|   

In [76]:
df.printSchema()

root
 |-- Ano: integer (nullable = true)
 |-- UF: integer (nullable = true)
 |-- CAPITAL: integer (nullable = true)
 |-- RM_RIDE: integer (nullable = true)
 |-- V1008: integer (nullable = true)
 |-- V1012: integer (nullable = true)
 |-- V1013: integer (nullable = true)
 |-- V1016: integer (nullable = true)
 |-- Estrato: integer (nullable = true)
 |-- UPA: integer (nullable = true)
 |-- V1022: integer (nullable = true)
 |-- V1023: integer (nullable = true)
 |-- V1030: integer (nullable = true)
 |-- V1031: double (nullable = true)
 |-- V1032: double (nullable = true)
 |-- posest: integer (nullable = true)
 |-- A001: integer (nullable = true)
 |-- A001A: integer (nullable = true)
 |-- A001B1: integer (nullable = true)
 |-- A001B2: integer (nullable = true)
 |-- A001B3: integer (nullable = true)
 |-- A002: integer (nullable = true)
 |-- A003: integer (nullable = true)
 |-- A004: integer (nullable = true)
 |-- A005: integer (nullable = true)
 |-- A006: integer (nullable = true)
 |-- A006A: 